# Tutorial 4: Nodes, Synchronization and Graph Validity

In this tutorial, we will discuss [nodes](https://eagerx.readthedocs.io/en/master/guide/api_reference/node/node.html) and the validity of the [graph](https://eagerx.readthedocs.io/en/master/guide/api_reference/graph/graph.html).

The following will be covered:
- Creating a node
- Adding a node to the graph
- Checking the validity of the graph
- Synchronization within EAGERx

In the remainder of this tutorial we will go more into detail on these concepts.

Furthermore, at the end of this notebook you will find an assignment.
For the assignment you will have to add/modify a couple of lines of code, which are marked by

```python

# START ASSIGNMENT [BLOCK_NUMBER]

# END ASSIGNMENT [BLOCK_NUMBER]
```

## Pendulum Swing-up

We will create an environment for solving the classic control problem of swinging up an underactuated pendulum, very similar to the [Pendulum-v0 environment](https://gym.openai.com/envs/Pendulum-v0/).
Our goal is to swing up this pendulum to the upright position and keep it there, while minimizing the velocity of the pendulum and the input voltage.

Since the dynamics of a pendulum actuated by a DC motor are well known, we can simulate the pendulum by integrating the corresponding ordinary differential equations (ODEs):


$\mathbf{x} = \begin{bmatrix} \theta \\ \dot{\theta} \end{bmatrix} \\ \dot{\mathbf{x}} = \begin{bmatrix} \dot{\theta} \\ \frac{1}{J}(\frac{K}{R}u - mgl \sin{\theta} - b \dot{\theta} - \frac{K^2}{R}\dot{\theta})\end{bmatrix}$

with $\theta$ the angle w.r.t. upright position, $\dot{\theta}$ the angular velocity, $u$ the input voltage, $J$ the inertia, $m$ the mass, $g$ the gravitational constant, $l$ the length of the pendulum, $b$ the motor viscous friction constant, $K$ the motor constant and $R$ the electric resistance.

## Notebook Setup

In order to be able to run the code, we need to install the *eagerx_tutorials* package and ROS.

In [1]:
try:
    import eagerx_tutorials
except ImportError:
    !{"pip install eagerx-tutorials  >> /tmp/eagerx_install.txt"}
if 'google.colab' in str(get_ipython()):
  !{"curl 'https://raw.githubusercontent.com/eager-dev/eagerx_tutorials/master/scripts/setup_colab.sh' > ~/setup_colab.sh"}
  !{"bash ~/setup_colab.sh"}

# Setup interactive notebook
# Required in interactive notebooks only.
from eagerx_tutorials import helper
helper.setup_notebook()
env = None

# Allows reloading of registered entites from changed files
# Required in interactive notebooks only.
%reload_ext autoreload
%autoreload 1

Not running on CoLab.
Execute ROS commands as "!...".
ROS noetic available.


## Let's get started

We start by importing the required packages and initializing EAGERx.

In [2]:
import eagerx
import eagerx_tutorials.pendulum  # Registers Pendulum
import eagerx_ode  # Registers OdeBridge

# Initialize eagerx (starts roscore if not already started.)
eagerx.initialize("eagerx_core")

... logging to /home/jelle/.ros/log/d734afb2-c7c2-11ec-ab25-bdefe663dbb0/roslaunch-jelle-Alienware-m15-R4-74513.log
started roslaunch server http://145.94.60.89:44455/
ros_comm version 1.15.14


SUMMARY

PARAMETERS
 * /rosdistro: noetic
 * /rosversion: 1.15.14

NODES

[INFO] [1651245318.393119]: Roscore cannot run as another roscore/master is already running. Continuing without re-initializing the roscore.


In this tutorial we will discuss the entity type [nodes](https://eagerx.readthedocs.io/en/master/guide/api_reference/node/node.html).
In the previous tutorials we have showed how to add [object](https://eagerx.readthedocs.io/en/master/guide/api_reference/object/index.html) entities to a [graph](https://eagerx.readthedocs.io/en/master/guide/api_reference/graph/graph.html).
Apart from objects, we can also add nodes to a graph.
The difference between nodes and objects, is that nodes are bridge-agnostic while objects have bridge-specific implementations.
Objects are entities that can sense or act in the environments.
Nodes on the other hand can only process data.
A node could for example contain a classifier, PID controller or some sort of signal filter.
While we refer to the incoming and outcoming components of objects as sensors and actuators, a node has inputs and outputs since it cannot act or sense in the environment.

In this tutorial we will create a moving average filter that will act as a low-pass filter on the voltage $u$.
This will prevent sending high frequency voltage signals to the DC motor of the pendulum, since they might damage the motor.
This node is not finished however.
In the assignment you will finalize the implementation of the moving average filter node.

We can create a node by inheriting from the class [Node](https://eagerx.readthedocs.io/en/master/guide/api_reference/node/node.html).
This class has the following abstract methods we need to implement:

- [spec()](https://eagerx.readthedocs.io/en/master/guide/api_reference/node/node.html#eagerx.core.entities.Node.spec): Specifies the parameters of the node.
- [initialize()](https://eagerx.readthedocs.io/en/master/guide/api_reference/node/node.html#eagerx.core.entities.Node.initialize): Initializes the node.
- [reset()](https://eagerx.readthedocs.io/en/master/guide/api_reference/node/node.html#eagerx.core.entities.Node.reset): Resets the node at the beginning of an episode.
- [callback()](https://eagerx.readthedocs.io/en/master/guide/api_reference/node/node.html#eagerx.core.entities.Node.callback): Called at the rate of the node.

In [3]:
%%writefile node.py

import eagerx
import eagerx.converters  # Registers space converters
from eagerx.utils.utils import Msg
from std_msgs.msg import Float32, Float32MultiArray


class MovingAverageFilter(eagerx.Node):
    @staticmethod
    @eagerx.register.spec("ExampleNode", eagerx.Node)
    def spec(
        spec,
        name: str,
        rate: float,
        n: int,
    ):
        """
        MovingAverage filter
        :param spec: Not provided by user.
        :param name: Node name
        :param rate: Rate at which callback is called.
        :param n: Window size of the moving average
        :return:
        """
        # Performs all the steps to fill-in the params with registered info about all functions.
        spec.initialize(MovingAverageFilter)

        # Modify default node params
        spec.config.name = name
        spec.config.rate = rate
        spec.config.process = eagerx.process.ENVIRONMENT
        spec.config.inputs = ["signal"]
        spec.config.outputs = ["filtered"]
        
        # Custom node params
        # START ASSIGNMENT 1.1

        # START ASSIGNMENT 1.1
        
        # Add space converters
        spec.inputs.signal.space_converter = eagerx.SpaceConverter.make("Space_Float32", -3, 3, dtype="float32")
        spec.outputs.filtered.space_converter = eagerx.SpaceConverter.make("Space_Float32MultiArray", [-3], [3], dtype="float32")
    
    # START ASSIGNMENT 1.2
    def initialize(self):
        pass
    # END ASSIGNMENT 1.2
    
    @eagerx.register.states()
    def reset(self):
        # START ASSIGNMENT 1.3
        pass
        # END ASSIGNMENT 1.3

    @eagerx.register.inputs(signal=Float32)
    @eagerx.register.outputs(filtered=Float32MultiArray)
    def callback(self, t_n: float, signal: Msg):
        data = signal.msgs[-1].data
        
        # START ASSIGNMENT 1.4
        filtered_data = data
        # END ASSIGNMENT 1.4
        
        return dict(filtered=Float32MultiArray(data=[filtered_data]))

Overwriting node.py


Making a node is very similar to making an object.

In [4]:
%aimport node
import node

# Define rate (depends on rate of ode)
rate = 30.0

# Make moving average filter
maf = eagerx.Node.make("ExampleNode", "filter", rate=rate, n=5)

# Make pendulum
pendulum = eagerx.Object.make("Pendulum", "pendulum", actuators=["u"], sensors=["theta", "dtheta", "image"], states=["model_state"])

Next, we will add the moving average filter and the pendulum to an empty graph.

In [5]:
# Initialize empty graph
graph = eagerx.Graph.create()

# Add pendulum and node to the graph
graph.add([pendulum, maf])

# Connect the moving average filter to an action
graph.connect(action="voltage", target=maf.inputs.signal)

# Connect the moving average filter to the pendulum
graph.connect(source=maf.outputs.filtered, target=pendulum.actuators.u)

# Connect theta and dtheta to observations
graph.connect(source=pendulum.sensors.theta, observation="angle")
graph.connect(source=pendulum.sensors.dtheta, observation="angular_velocity")

# START ASSIGNMENT 2.1

# END ASSIGNMENT 2.1

# Render image
graph.render(source=pendulum.sensors.image, rate=rate)

# Make OdeBridge
bridge = eagerx.Bridge.make("OdeBridge", rate=rate)

Using the [*eagerx_gui* package](https://github.com/eager-dev/eagerx_gui), we see that the graph looks as follows:


```python
graph.gui()
```

<img src="./figures/tutorial_4_gui.svg" width=720>

In [6]:
import numpy as np
from typing import Dict
import stable_baselines3 as sb
from eagerx.wrappers import Flatten


# Define step function
def step_fn(prev_obs: Dict[str, np.ndarray], obs: Dict[str, np.ndarray], action: Dict[str, np.ndarray], steps: int):
    
    # Get angle and angular velocity
    # Take first element because of window size (covered in other tutorial)
    th = obs["angle"][0] 
    thdot = obs["angular_velocity"][0]
    
    # Convert from numpy array to float
    u = float(action["voltage"])
    
    # Calculate cost
    # Penalize angle error, angular velocity and input voltage
    cost = th**2 + 0.1 * thdot**2 + 0.001 * u**2  
    
    # Determine when is the episode over
    # currently just a timeout after 100 steps
    done = steps > 100
    
    # Set info, tell the algorithm the termination was due to a timeout
    # (the episode was truncated)
    info = {"TimeLimit.truncated": steps > 100}
    
    return obs, -cost, done, info

# Initialize Environment
env = eagerx.EagerxEnv(name="PendulumEnv", rate=rate, graph=graph, bridge=bridge, step_fn=step_fn)

# Toggle render
env.render("human")

# Stable Baselines3 expects flattened actions & observations
# Convert observation and action space from Dict() to Box()
env = Flatten(env)

# Initialize learner
model = sb.SAC("MlpPolicy", env, verbose=1)

# Train for 1 minute (sim time)
model.learn(total_timesteps=int(60 * rate))

env.shutdown()

[INFO] [1651245319.137477]: Node "/PendulumEnv/env/supervisor" initialized.
[INFO] [1651245319.281076]: Node "/PendulumEnv/bridge" initialized.
[INFO] [1651245319.405121]: Node "/PendulumEnv/environment" initialized.
[INFO] [1651245319.456592]: Node "/PendulumEnv/env/render" initialized.
[INFO] [1651245319.534099]: Node "/PendulumEnv/filter" initialized.
[INFO] [1651245319.580993]: Node "/PendulumEnv/pendulum/theta" initialized.
[INFO] [1651245319.605309]: Node "/PendulumEnv/pendulum/dtheta" initialized.
[INFO] [1651245319.681458]: START RENDERING!
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
[INFO] [1651245319.703572]: Adding object "pendulum" of type "Pendulum" to the simulator.
[INFO] [1651245319.720563]: Node "/PendulumEnv/pendulum/x" initialized.
[INFO] [1651245319.730005]: [pendulum/image] START RENDERING!
[INFO] [1651245319.736262]: Node "/PendulumEnv/pendulum/image" initialized.
[INFO] [1651245319.751752]: Node "/PendulumEnv/pen

# Assignment

In this assignment you finalize the implementation of the moving average filter node.
Furthermore, the Markov property will be violated after implementing the moving average filter.
You will restore the Markov property, while considering the graph validity.

For this assignment, you will need to modify or add some lines of code in the cells above.
These lines are indicated by the following comments:

```python
# START ASSIGNMENT [BLOCK_NUMBER]

# END ASSIGNMENT [BLOCK_NUMBER]
```

However, feel free to play with the other code as well if you are interested.
We recommend you to restart and run all code after each section (in Colab there is the option *Restart and run all* under *Runtime*).


## 1. Finalize the moving average filter

### Add your code to the following blocks: 

1.1  
1.2  
1.3  

## 2. Restore the Markov property


After this the graph should look as follows:

<img src="./figures/tutorial_42_gui.svg" width=720>

### Add your code to the following blocks: 

2.1  